In [1]:
!pip install pytesseract
!apt-get install poppler-utils 
!pip install pdf2image
!pip install pytesseract
!pip install opencv-python
!pip install pandas

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
     |████████████████████████████████| 49.9 MB 149 kB/s eta 0:00:01


In [2]:
!which tesseract

/usr/bin/tesseract


In [3]:
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import image_to_string
from IPython.display import Image as pim, display
import csv
import os
import pandas as pd
from multiprocessing import Pool

In [4]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [5]:
CORES = 5
DIRECTORY_PATH = "../../../jalend/Electoral_Data_2021/Sikkim/Sikkim/"
DOWNLOAD_LOCATION = "../ParsedData/Sikkim/"
LOG_DIR = "../ParsedData/Sikkim/logs/"
TEMP_DIR = "../ParsedData/Sikkim/temp_files/"

In [6]:
def convert_to_csv(filename, data, header):
#     with open(LOG_DIR+'log_test.txt', 'a') as log_file:
#         log_file.write(f"PID-{os.getpid()} converting {filename} data to csv!!\n")
    df = pd.DataFrame(data, columns = header)
    filename = filename.replace(".pdf","")
    for i, char in enumerate(filename):
        if char == '/':
            break
    filename = filename[i:]
    path = DOWNLOAD_LOCATION+filename+"_parsed.csv"
    with open(LOG_DIR+'log_test_new_1.txt', 'a') as log_file:
        log_file.write(f"PID-{os.getpid()} at {path} !!\n")
    df.to_csv(path)

def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file)

def ocr_core(file):
    text = pytesseract.image_to_string(file)
    return text

def get_name(line_str, rem_str):
    line_str = line_str.replace(rem_str, "")
    line_str = line_str.strip()
    # Assuming first letter of a name to be alphabet
    i = 0
    for i, char in enumerate(line_str):
      if char.isalpha():
        break
    line_str = line_str[i:]
    return line_str

def parse_info(elem, data):
    husband_name = ""
    father_name = ""
    mother_name = ""
    house_number = ""
    name = ""
    age = 0
    gender = ""
    for line in elem.split('\n'):
      # print(line)
      if "Husband's Name" in line:
        husband_name = get_name(line, "Husband's Name")
      elif "Father's Name" in line:
        father_name = get_name(line, "Father's Name")
      elif "Mother's Name" in line:
        mother_name = get_name(line, "Mother's Name")
      elif "House Number" in line:
        house_number = get_name(line, "House Number")
      elif "Name" in line:
        name = get_name(line, "Name")
      elif "Age" in line:
        try:
          age = int(line[5:7])
        except:
          pass
        # Considering only 2 genders [for now]
        if "FEMALE" in line:
            gender = "FEMALE"
        elif "MALE" in line:
            gender = "MALE"
      elif "FEMALE" in line:
        gender = "FEMALE"
      elif "MALE" in line:
        gender = "MALE"
    if name == "":
      return data
    arr = [name, father_name, husband_name, mother_name, house_number, gender, age]
    # print(husband_name,":",father_name,":",mother_name,":",house_number,":", name,":",str(age),":",gender)
    data.append(arr)
    return data

def extract_to_csv(img, data):
    pid = str(os.getpid())
    with open(LOG_DIR+'log_test_new_1.txt', 'a') as log_file:
        log_file.write(f"{pid},")
    
    image_name = pid+"sample_sikkim.jpeg"  
    img.save(TEMP_DIR+image_name, "JPEG")
    # Extract data from page and delete
    # Highlight the region that we want to extract
    upper = 140
    lower = 333
    d1 = lower-upper
    for i in range(1,11):
      right = 565
      left = 50
      d = right - left
      for j in range(1,4):
          im = Image.open(TEMP_DIR+image_name)
          im_crop = im.crop((left, upper, right, lower))
          im_crop.save(TEMP_DIR+pid+'test_sikkim.png', quality=95)
          # display(pim('test.png'))
          text = ocr_core(Image.open(TEMP_DIR+pid+'test_sikkim.png'))
          data = parse_info(text, data)
          # print(text)          
          left = left+d
          right = right+d
      upper = upper + d1
      lower = lower + d1
    return data

def convert_pdf_to_csv(pdf_file):
    header = ['name', 'father_name', 'husband_name', 'mother_name', 'house_number', 'gender', 'age']
    data = []
    loc = DIRECTORY_PATH
    images = pdf_to_img(loc+pdf_file)
    
    with open(LOG_DIR+'log_test_new_1.txt', 'a') as log:
        log.write(f"PID- {os.getpid()} : is converting {pdf_file} ==== \n")
    for img in images[2:-1]:
        data = extract_to_csv(img, data)
        
    with open(LOG_DIR+'log_test_new_1.txt', 'a') as log:
        log.write(f"\n")
    
    try:
        convert_to_csv(pdf_file, data, header)
#         with open(LOG_DIR+'log_test.txt', 'a') as log_file:
#             log_file.write(f"{pdf_file} is converted to csv by PID-{os.getpid()}\n")
    except:
        with open(LOG_DIR+'log_test_new_1.txt', 'a') as log_file:
            log_file.write(f"Unable to parse {pdf_file} by {os.getpid()}\n")

# Convert all the pdfs present in a folder to a csv
def extract_data(files):
    for file in files:
        convert_pdf_to_csv(file)

In [7]:
# Get all files in a directory
def get_files(location):
    direc = os.listdir(location)
    downloaded = os.listdir(DOWNLOAD_LOCATION)
    files_present = []
    for elem in downloaded:
        if ".csv" in elem:
            for i, char in enumerate(elem):
                if char == '_':
                    break
            elem = elem[:i]
            elem = elem + ".pdf"
            print(elem)
            files_present.append(elem)
        
    arr = []
    for elem in direc:
        if '.' not in elem:
            files = os.listdir(location+elem+'/')
            for file in files:
              if ".pdf" in file and file not in files_present:
                arr.append(elem+'/'+file)
    return arr, files_present

In [8]:
files, pre = get_files(DIRECTORY_PATH)

S21A19P19.pdf
S21A32P40.pdf
S21A20P1.pdf
S21A21P18.pdf
S21A1P17.pdf
S21A27P6.pdf
S21A16P12.pdf
S21A14P8.pdf
S21A11P2.pdf
S21A30P20.pdf
S21A19P15.pdf
S21A11P11.pdf
S21A32P23.pdf
S21A17P7.pdf
S21A2P7.pdf
S21A23P13.pdf
S21A7P10.pdf
S21A10P17.pdf
S21A25P6.pdf
S21A7P19.pdf
S21A5P14.pdf
S21A13P11.pdf
S21A24P3.pdf
S21A13P1.pdf
S21A21P15.pdf
S21A5P17.pdf
S21A28P5.pdf
S21A7P2.pdf
S21A9P12.pdf
S21A7P17.pdf
S21A28P2.pdf
S21A19P21.pdf
S21A21P4.pdf
S21A21P7.pdf
S21A32P43.pdf
S21A3P22.pdf
S21A19P3.pdf
S21A9P11.pdf
S21A5P4.pdf
S21A16P14.pdf
S21A25P2.pdf
S21A14P2.pdf
S21A20P18.pdf
S21A17P11.pdf
S21A17P2.pdf
S21A17P13.pdf
S21A9P5.pdf
S21A15P14.pdf
S21A13P14.pdf
S21A7P1.pdf
S21A14P3.pdf
S21A26P3.pdf
S21A26P2.pdf
S21A15P1.pdf
S21A31P2.pdf
S21A32P11.pdf
S21A21P17.pdf
S21A20P2.pdf
S21A32P3.pdf
S21A30P7.pdf
S21A1P14.pdf
S21A30P18.pdf
S21A11P1.pdf
S21A11P9.pdf
S21A20P8.pdf
S21A9P1.pdf
S21A24P15.pdf
S21A19P26.pdf
S21A17P12.pdf
S21A9P6.pdf
S21A4P10.pdf
S21A28P6.pdf
S21A23P7.pdf
S21A15P8.pdf
S21A11P14.pdf
S21A2

In [9]:
print(len(files) , len(pre))

50 564


In [10]:
# if __name__ == '__main__':
with Pool(CORES) as p:
    p.map(convert_pdf_to_csv, files)

In [ ]:
extract_data(files)